TODOs:

- [ ] Finde in Airports_Rail_Access die Angabe zur City–Airport-Zeit.
- [ ] Extrahiere daraus die Fahrzeit in Minuten.
- [ ] Behandle Airports ohne Angaben.
- [ ] Baue eine Tabelle: Airport ↔ City-Airport-Time.
- [ ] Nimm aus Alternative_Rail_Routes die Flugzeit.
- [ ] Mappe Origin/Destination-Airport auf City–Airport-Time.
- [ ] Berechne Total Air Travel Duration.
- [ ] Interpretation + Ergebnis.

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

# Q3: 

## How can we estimate the typical travel time between the city’s central rail station and its nearest major airport, and how should this time be incorporated into the total air travel duration?

In [3]:
# Path to the data directory
data_path = "../Data"

# List all xlsx files in the data directorya
xlsx_files = [f for f in os.listdir(data_path) if f.endswith(".xlsx")]

In [4]:
# Dictionary to hold dataframes
dataframes = {}

for file in xlsx_files:
    file_path = os.path.join(data_path, file)
    if file == "Airports_Rail_Access.xlsx":
        df = pd.read_excel(file_path, header=1, sheet_name=0)  # multi-index columns
    else:
        df = pd.read_excel(file_path, sheet_name=0)  # only 1 sheet per file
    dataframes[file] = df
    print(f"Loaded: {file} — Shape: {df.shape} - Missing values: {df.isnull().sum().sum()}")

Loaded: Airports_Rail_Access.xlsx — Shape: (318, 25) - Missing values: 309
Loaded: Candidate_Rail_Routes.xlsx — Shape: (8612, 11) - Missing values: 3172
Loaded: Alternative_Rail_Routes_Unidirectional.xlsx — Shape: (7425, 30) - Missing values: 27010
Loaded: Alternative_Rail_Routes_Bidirectional.xlsx — Shape: (3814, 16) - Missing values: 5148
Loaded: Airports_Indicators.xlsx — Shape: (58, 10) - Missing values: 0


## Subquestion:
## How can one estimate the typical travel time between City Central Station and the airport?

### Analyse Airports_Rail_Access.xlsx

In [ ]:
dataframes_list = list(dataframes.items())

# Airports_Rail_Access.xlsx
filename_ARA, df_ARA = dataframes_list[0]

# Important columns
df_ARA_clean = df_ARA[["Airport", "IATA Code", "Train connection to City Station"]].copy()

# Rename columns for easier access
df_ARA_clean = df_ARA_clean.rename(columns={
    "IATA Code": "iata_code",
    "Train connection to City Station": "connection_text"
})

# Extract city airport time in minutes using regex
df_ARA_clean["city_airport_time_min"] = df_ARA_clean["connection_text"].str.extract(r"(\d+)\s*min").astype("Int64")
df_ARA_clean.drop(columns=["connection_text"], inplace=True)

## Analyse Alternative_Rail_Routes_Bidirectional.xlsx

In [39]:
# Alternative_Rail_Routes_Bidirectional.xlsx
filename_ARRB, df_ARRB = dataframes_list[3]

df_ARRB.head()

,Bidirectional Route Name,Origin,Destination,Origin Coordinates,Destination Coordinates,Rail Theoretical Travel Time,Rail Service Travel Time,Air Travel Time,Valued Air Travel Time,Passenger Flows,Theoretical ARISP,Service ARISP,Theoretical Time Difference,Service Time Difference,Theoretical Time Increase,Service Time Increase
0,A Coruña-AGP,A Coruña,AGP,"43.352681,-8.409651","36.677749,-4.490162",466.960144,471.064213,100.010468,162.016959,3613,1253.570094,1242.648572,-304.943185,-309.047254,188.216831,190.749942
1,A Coruña-BCN,A Coruña,BCN,"43.352681,-8.409651","41.304141,2.073126",456.163626,867.916176,104.935906,169.996167,34310,12786.132358,6720.197937,-286.167459,-697.920009,168.337595,410.550438
2,A Coruña-GVA,A Coruña,GVA,"43.352681,-8.409651","46.231334,6.110271",813.577093,964.708150,135.000000,218.700000,908,244.082094,205.844224,-594.877093,-746.008150,272.005987,341.110265
3,A Coruña-LGW,A Coruña,LGW,"43.352681,-8.409651","51.156735,-0.160922",885.255404,1250.218575,119.621596,193.786986,4996,1093.650236,774.392415,-691.468418,-1056.431589,356.818811,545.150947
4,A Coruña-MAD,A Coruña,MAD,"43.352681,-8.409651","40.492435,-3.592839",317.769684,273.757594,76.021220,123.154376,184030,71322.410288,82788.935567,-194.615308,-150.603218,158.025492,122.288158


In [38]:
# Important columns
df_ARRB_clean = df_ARRB[["Origin", "Destination", "Air Travel Time"]].copy()

# Rename columns for easier access
df_ARRB_clean = df_ARRB_clean.rename(columns={
    "Origin": "origin",
    "Destination": "destination",
    "Air Travel Time": "air_travel_time_min"
})

df_ARRB_clean

,origin,destination,air_travel_time_min
0,A Coruña,AGP,100.010468
1,A Coruña,BCN,104.935906
2,A Coruña,GVA,135.000000
3,A Coruña,LGW,119.621596
4,A Coruña,MAD,76.021220
...,...,...,...
3809,Warsaw,ZRH,118.072943
3810,Westerland,ZRH,99.209522
3811,Wrocław,ZRH,96.747235
3812,XRY,Zurich,165.000000
